# Simple Demo
A few of the Dash examples packaged together in a simple Jupyter notebook to make interactive testing easier

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [2]:
from IPython import display
def show_app(app, port = 9999, width = 700, height = 350, offline = False):
    url = 'http://localhost:%d' % port
    iframe = '<iframe src="{url}" width={width} height={height}></iframe>'.format(url = url, 
                                                                                  width = width, 
                                                                                  height = height)
    display.display_html(iframe, raw = True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    return app.run_server(debug=False, # needs to be false in Jupyter
                          host = '0.0.0.0',
                          port=port)

In [4]:
app_simple = dash.Dash()

app_simple.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

In [5]:
show_app(app_simple)

 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)
172.17.0.1 - - [05/Oct/2017 17:41:08] "GET / HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:41:09] "GET /_dash-layout HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:41:09] "GET /_dash-dependencies HTTP/1.1" 200 -


In [6]:
import pandas as pd

df = pd.read_csv(
    'https://gist.githubusercontent.com/chriddyp/'
    'c78bf172206ce24f77d6363a2d754b59/raw/'
    'c353e8ef842413cae56ae3920b8fd78468aa4cb2/'
    'usa-agricultural-exports-2011.csv')

def generate_table(dataframe, max_rows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )

app_table = dash.Dash()

app_table.layout = html.Div(children=[
    html.H4(children='US Agriculture Exports (2011)'),
        
    generate_table(df)
])

In [7]:
show_app(app_table)

 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)
172.17.0.1 - - [05/Oct/2017 17:41:32] "GET / HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:41:32] "GET /_dash-layout HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:41:32] "GET /_dash-dependencies HTTP/1.1" 200 -


In [17]:
import pandas as pd

df = pd.read_csv(
    'https://gist.githubusercontent.com/chriddyp/'
    'c78bf172206ce24f77d6363a2d754b59/raw/'
    'c353e8ef842413cae56ae3920b8fd78468aa4cb2/'
    'usa-agricultural-exports-2011.csv')


app_pd_table = dash.Dash()

app_pd_table.layout = html.Div(children=[
    html.H4(children='US Agriculture Exports (2011)'),
    html.Source(src = df.to_html())
])

In [18]:
show_app(app_pd_table)

 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)
172.17.0.1 - - [05/Oct/2017 15:52:37] "GET / HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 15:52:38] "GET /_dash-layout HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 15:52:38] "GET /_dash-dependencies HTTP/1.1" 200 -


In [8]:
app_inter = dash.Dash()

app_inter.layout = html.Div([
    dcc.Input(id='my-id', value='initial value', type="text"),
    html.Div(id='my-div')
])

@app_inter.callback(
    Output(component_id='my-div', component_property='children'),
    [Input(component_id='my-id', component_property='value')]
)
def update_output_div(input_value):
    return 'You\'ve entered "{}"'.format(input_value)

In [9]:
show_app(app_inter)

 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)
172.17.0.1 - - [05/Oct/2017 17:41:47] "GET / HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:41:48] "GET /_dash-layout HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:41:48] "GET /_dash-dependencies HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:41:48] "POST /_dash-update-component HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:41:52] "POST /_dash-update-component HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:41:52] "POST /_dash-update-component HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:41:52] "POST /_dash-update-component HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:41:52] "POST /_dash-update-component HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:41:53] "POST /_dash-update-component HTTP/1.1" 200 -


# Incorporating Matplotlib Plots
We convert the figure to a data-uri to show it

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO
import base64
def fig_to_uri(in_fig, close_all=True, **save_args):
    # type: (plt.Figure) -> str
    """
    Save a figure as a URI
    :param in_fig:
    :return:
    """
    out_img = BytesIO()
    in_fig.savefig(out_img, format='png', **save_args)
    if close_all:
        in_fig.clf()
        plt.close('all')
    out_img.seek(0)  # rewind file
    encoded = base64.b64encode(out_img.read()).decode("ascii").replace("\n", "")
    return "data:image/png;base64,{}".format(encoded)
app_iplot = dash.Dash()

app_iplot.layout = html.Div([
    dcc.Input(id='plot_title', value='Type title...', type="text"),
    dcc.Slider(
        id='box_size',
        min=1,
        max=10,
        value=4,
        step=1,
        marks=list(range(0, 10))
    ),
    html.Div([html.Img(id = 'cur_plot', src = '')],
             id='plot_div')
])

@app_iplot.callback(
    Output(component_id='cur_plot', component_property='src'),
    [Input(component_id='plot_title', component_property='value'), Input(component_id = 'box_size', component_property='value')]
)
def update_graph(input_value, n_val):
    fig, ax1 = plt.subplots(1,1)
    np.random.seed(len(input_value))
    ax1.matshow(np.random.uniform(-1,1, size = (n_val,n_val)))
    ax1.set_title(input_value)
    out_url = fig_to_uri(fig)
    return out_url

In [11]:
show_app(app_iplot)

 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)
172.17.0.1 - - [05/Oct/2017 17:42:29] "GET / HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:42:29] "GET / HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:42:30] "GET /_dash-layout HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:42:30] "GET /_dash-dependencies HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:42:30] "POST /_dash-update-component HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:42:31] "GET /_dash-layout HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:42:31] "GET /_dash-dependencies HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:42:31] "POST /_dash-update-component HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:42:31] "GET /favicon.ico HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:42:34] "POST /_dash-update-component HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:42:37] "POST /_dash-update-component HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:42:37] "POST /_dash-update-component HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2

# Interactive 2D Plots
Here you can click on the plot and it will be updated based on your click

In [12]:
import plotly.graph_objs as go
import plotly.plotly as py
import numpy as np

app_iplotly = dash.Dash()

app_iplotly.layout = html.Div([
    html.Div([
            html.Div(id='click_info'),
             dcc.Graph(id='cur_plot', animate=False)
            ],
             id='plot_div')
])


def _simple_image(xval, yval, as_uri = True):
    print(xval, yval, 'creating')
    lin_x = np.arange(512)
    lin_y = np.arange(512)
    xx, yy = np.meshgrid(lin_y, lin_x)
    out_mat = np.sqrt(np.square(xx-xval)+np.square(yy-yval))
    if as_uri:
        fig, ax1 = plt.matshow(out_mat)
        out_uri = fig_to_uri(fig)
        return dict(source = out_uri,
                    x = lin_x.min(),
                    sizex = lin_x.max()-lin_x.min(),
                    y = lin_y.max(),
                    sizey = lin_y.max() - lin_y.min(),
                    xref= "x",
                    yref= "y",
                    sizing= "stretch",
                    opacity= 0.5,
                    layer= "below"
                   )
    else:
        return out_mat


def update_graph(x_val, y_val):
    traces = [go.Heatmap(z=_simple_image(x_val, y_val, as_uri = False),
                        #colorscale='Viridis'
                        )]
    if (x_val>0) or (y_val>0):
        traces += [go.Scatter(x = [x_val], y = [y_val])]
    
    layout = go.Layout(
        #images= [_simple_image(x_val,y_val)],
        annotations=[
        dict(                            # all "annotation" attributes: /python/reference/#layout-annotations
            text="Current Position ({}, {})".format(x_val, y_val),    # /python/reference/#layout-annotations-text
            x=x_val,                         # /python/reference/#layout-annotations-x
            xref="y",                # /python/reference/#layout-annotations-xref
            y=y_val,                         # /python/reference/#layout-annotations-y
            yref="y"                 # /python/reference/#layout-annotations-yref
        )
    ]
                      )
    return dict(data = traces, layout=layout)

def get_hover_info(hover_data):
    if hover_data is not None:
        clicks = hover_data.get('points', [])
        if len(clicks)>0:
            x, y = clicks[0].get('x', 0), clicks[0].get('y', 0)
            print(x,y, 'hovering')
            return x,y
    return 0,0

@app_iplotly.callback(
    Output(component_id='cur_plot', component_property='figure'),
    [Input(component_id='cur_plot', component_property='clickData')]
)
def update_click_info(hover_data):
    return update_graph(*get_hover_info(hover_data))

In [ ]:
show_app(app_iplotly)

 * Running on http://0.0.0.0:9999/ (Press CTRL+C to quit)
172.17.0.1 - - [05/Oct/2017 17:42:52] "GET / HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:42:53] "GET /_dash-layout HTTP/1.1" 200 -
172.17.0.1 - - [05/Oct/2017 17:42:53] "GET /_dash-dependencies HTTP/1.1" 200 -


0 0 creating


172.17.0.1 - - [05/Oct/2017 17:42:54] "POST /_dash-update-component HTTP/1.1" 200 -


228 322 hovering
228 322 creating


172.17.0.1 - - [05/Oct/2017 17:42:58] "POST /_dash-update-component HTTP/1.1" 200 -


...

172.17.0.1 - - [05/Oct/2017 17:43:10] "POST /_dash-update-component HTTP/1.1" 200 -


96 326 hovering
96 326 creating


172.17.0.1 - - [05/Oct/2017 17:43:11] "POST /_dash-update-component HTTP/1.1" 200 -


367 252 hovering
367 252 creating


172.17.0.1 - - [05/Oct/2017 17:51:20] "POST /_dash-update-component HTTP/1.1" 200 -
